In [7]:
from langchain_community.chat_models import ChatZhipuAI

ZHIPUAI_API_KEY = "0f2f6aca4fb384e90505b066a1ddf204.hM6JYEdsgvTYLxWz"
llm = ChatZhipuAI(
    temperature=0.1,
    api_key=ZHIPUAI_API_KEY,
    model_name="glm-4",
)

from langchain_community.utilities import SQLDatabase
db = SQLDatabase.from_uri("sqlite:///G:/Python Project/llm+sql/ali_langchain.db")

def get_schema(_):
    return db.get_table_info()
    
def run_query(query):
    return db.run(query)

result = run_query("SELECT COUNT(*) FROM alidata")

In [8]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda

template_sql = (
    "请通过写SQL代码来回答问题，并且需要基于如下数据库信息：{info}\n"
    "需要回答的问题是：{question}\n"
    "注意：仅需要通过SQL代码回答，不需要文字解释。\n"
    "代码形式如下：\n"
    "```sql\n"
    "{sql}\n"
    "```"
)
t_sql=PromptTemplate.from_template(template_sql)

def get_sql(x):
    return x.split("```sql")[1].split("```")[0]

chain_sql=({"info":get_schema,"question":RunnablePassthrough()}
           |t_sql
           |llm
           |StrOutputParser()
           |RunnableLambda(get_sql))

template_sql0="请通过综合如下的数据库信息，问题，sql代码，sql代码的执行结果给出问题的自然语言回答：\n 数据库信息{info} \n 需要回答问题是：{question} \n sql代码：{query} \n sql代码执行结果: {res}"

prompt=PromptTemplate.from_template(template_sql0)

chain_sql0=({"info":get_schema,
             "question":RunnablePassthrough(),
             "query":chain_sql
             }|RunnablePassthrough.assign(res=lambda x: run_query(x["query"]))
             |prompt
             |llm
             |StrOutputParser())

In [9]:
chain_sql.invoke("记录数最多的五个brand是")

KeyError: "Input to PromptTemplate is missing variables {'sql'}.  Expected: ['info', 'question', 'sql'] Received: ['info', 'question']"

In [6]:
chain_sql0.invoke("记录数最多的五个brand是")

KeyError: "Input to PromptTemplate is missing variables {'sql'}.  Expected: ['info', 'question', 'sql'] Received: ['info', 'question']"